# 第10章: ベクトル空間法 (II)

第10章では，前章に引き続き単語ベクトルの学習に取り組む．

In [1]:
from gensim.models import Word2Vec

# 90. word2vecによる学習
81で作成したコーパスに対してword2vecを適用し，単語ベクトルを学習せよ．さらに，学習した単語ベクトルの形式を変換し，86-89のプログラムを動かせ．

In [2]:
# https://rare-technologies.com/word2vec-tutorial/
with open('81.txt', 'r') as f:
    corpus = [f.read().split()]
model = Word2Vec(corpus, min_count=1)    

In [3]:
# 86 ベクトル表示
model['United_States']

array([ 0.00338275, -0.00363003,  0.00424971, -0.00084945, -0.00309413,
        0.00417765, -0.00094517,  0.00034748, -0.00229835, -0.00194972,
       -0.00405894,  0.00284101,  0.00090113,  0.00282354,  0.00085077,
       -0.00102706,  0.0029569 ,  0.00481773, -0.00333951, -0.00494955,
        0.00058217,  0.00133022, -0.002459  , -0.00013361, -0.00170235,
       -0.00103415, -0.00399301,  0.00498326, -0.00183375, -0.00091982,
       -0.00339756, -0.00176154,  0.00534766, -0.00319785,  0.00101649,
        0.0050673 ,  0.00236146, -0.00278086,  0.00312177,  0.00350982,
        0.00207829,  0.00453756,  0.00366913,  0.00494404, -0.00013967,
        0.00314772, -0.0006055 ,  0.00364694, -0.00178049, -0.00211092,
        0.00411564,  0.00199139,  0.0010052 , -0.00368206,  0.00217837,
        0.00276143,  0.00230237,  0.00220139, -0.00330923,  0.00092225,
        0.00108726,  0.00348572, -0.00146816,  0.00313841, -0.00247652,
        0.00102388, -0.00015509,  0.00149702,  0.00440409, -0.00

In [4]:
# 87 コサイン類似度 
model.similarity("United_States", "U.S")

-0.10478716769041592

In [5]:
# 88 類似度 top10
model.most_similar(positive=['England'], topn=10)

[('mainly', 0.4305400550365448),
 ('double-helical', 0.42140504717826843),
 ('leaked', 0.41813257336616516),
 ('Alentejo', 0.415567547082901),
 ('reconciling', 0.40542083978652954),
 ('fermée', 0.40274858474731445),
 ('Wickersham', 0.40134602785110474),
 ('Krummenau', 0.40052151679992676),
 ('Wan', 0.39969754219055176),
 ('Izyaslavich', 0.396148145198822)]

In [6]:
# 89 アナロジー top10
model.most_similar(positive=['Spain', 'Athens'], negative=['Madrid'], topn=10)

[('"Afrikakorps"', 0.4499354064464569),
 ('Krom', 0.44138073921203613),
 ('midnight.', 0.42218679189682007),
 ('ligature', 0.4191877245903015),
 ('Seddu', 0.4180520176887512),
 ('-Don', 0.41638216376304626),
 ('93-page', 0.4152953028678894),
 ('Publishings', 0.41490593552589417),
 ('Ostrivky', 0.4095712900161743),
 ('gunniella', 0.4037185609340668)]

# 91. アナロジーデータの準備
単語アナロジーの評価データをダウンロードせよ．このデータ中で": "で始まる行はセクション名を表す．例えば，": capital-common-countries"という行は，"capital-common-countries"というセクションの開始を表している．ダウンロードした評価データの中で，"family"というセクションに含まれる評価事例を抜き出してファイルに保存せよ．

In [7]:
with open('questions-words.txt', 'r') as f:
    sections = f.read().split(':')
    for section in sections:
        if 'family' in section.split('\n')[0]:
            family = "\n".join(section.split('\n')[1:])
with open('family.txt', 'w') as f:
    f.write(family)

# 92. アナロジーデータへの適用
91で作成した評価データの各事例に対して，vec(2列目の単語) - vec(1列目の単語) + vec(3列目の単語)を計算し，そのベクトルと類似度が最も高い単語と，その類似度を求めよ．求めた単語と類似度は，各事例の末尾に追記せよ．このプログラムを85で作成した単語ベクトル，90で作成した単語ベクトルに対して適用せよ．

In [8]:
with open('family.txt', 'r') as f:
    family = [x[:-1].split() for x in f.readlines()]
len(family)

506

In [9]:
def check_contain(words):
    for w in words:
        if not w in model:
            return False 
    return True

In [10]:
with open('92.txt', 'w') as f:
    for line in family:
        c1, c2, c3, c4 = line
        nomatch = model.doesnt_match(line)
        if check_contain(line):
            w, p = model.most_similar(positive=[c2, c3], negative=[c1], topn=1)[0]
            line = str(w) + "\t" + str(p) + "\n"
        else:
            line = "NoMatch\n"
        f.write(line)

# 93. アナロジータスクの正解率の計算
92で作ったデータを用い，各モデルのアナロジータスクの正解率を求めよ．

In [11]:
count93 = 0
with open('92.txt', 'r') as f:
    for i, line in enumerate(f.readlines()):
        w = line[:-1].split()[0]
        ans_w = family[i][3]
        if w != "NoMatch" and w == ans_w:
            count93 += 1
            print("match!!!!!")
count93 / len(family)

0.0

# 94. WordSimilarity-353での類似度計算
The WordSimilarity-353 Test Collectionの評価データを入力とし，1列目と2列目の単語の類似度を計算し，各行の末尾に類似度の値を追加するプログラムを作成せよ．このプログラムを85で作成した単語ベクトル，90で作成した単語ベクトルに対して適用せよ．

In [55]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
class W2V:
    def __init__(self):
        with open('Xpca.csv', 'r') as f:
            self.Xpca = np.array([np.array(line[:-1].split(','), dtype=np.float64) for line in f.readlines()])
        with open('dictionary.txt', 'r') as f:
            self.dictionary = np.array([x[:-1] for x in f.readlines()])

    def vec(self, word):
        dictionary = self.dictionary
        i = np.where(dictionary == word)[0][0]
        return self.Xpca[i].reshape(1, -1)

    def get_word_cs(self, word1, word2):
        return cosine_similarity(self.vec(word1), self.vec(word2))[0][0]
    
    def get_top10_w(self, word):
        lst=[]
        for i, w in enumerate(self.Xpca[:1000]):
            cs = cosine_similarity(self.vec(word), w.reshape(1, -1))[0][0]
            lst.append((self.dictionary[i], cs))
        return pd.DataFrame(lst).sort_values(1, ascending=False).head(10)
    
    def get_top10_v(self, vec):
        lst=[]
        for i, w in enumerate(self.Xpca[:1000]):
            cs = cosine_similarity(vec, w.reshape(1, -1))
            lst.append((self.dictionary[i], cs[0][0]))
        df = pd.DataFrame(lst)
        return df.sort_values(1, ascending=False).head(10)

In [56]:
w2v = W2V()

In [57]:
w2v.get_top10_w("England")

,0,1
47,events,0.806443
616,areas,0.752089
736,without,0.476834
207,3.84%,0.143077
63,seceded,0.115013
292,propitious,0.096278
177,Mosson,0.094246
301,own,0.089700
127,PEN/Laura,0.077884
174,Town.,0.067260


In [27]:
with open('combined.csv', 'r') as f:
    combined = f.readlines()[1:]
with open('94.csv', 'w') as f:
     for line in combined:
            w1, w2, p = line.split(',')
            if check_contain([w1, w2]):
                s90 = model.similarity(w1, w2)
            else:
                s90 = 0
            l = line[:-1] + ',' + str(s90) + '\n'
            f.write(l)

# 95. WordSimilarity-353での評価
94で作ったデータを用い，各モデルが出力する類似度のランキングと，人間の類似度判定のランキングの間のスピアマン相関係数を計算せよ．

# 96. 国名に関するベクトルの抽出
word2vecの学習結果から，国名に関するベクトルのみを抜き出せ．

# 97. k-meansクラスタリング
96の単語ベクトルに対して，k-meansクラスタリングをクラスタ数k=5k=5として実行せよ．

# 98. Ward法によるクラスタリング
96の単語ベクトルに対して，Ward法による階層型クラスタリングを実行せよ．さらに，クラスタリング結果をデンドログラムとして可視化せよ．

# 99. t-SNEによる可視化
96の単語ベクトルに対して，ベクトル空間をt-SNEで可視化せよ．